In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Explore the Data

Outline:

1. Download the dataset from Kaggle to Colab
2. Explore the data using Pandas
3. Create a small working sample

Explore the Data using Pandas

In [ ]:
import pandas as pd
import numpy as np
raw_df=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
raw_df

In [ ]:
sincere_df = raw_df[raw_df.target == 0]

In [ ]:
sincere_df.question_text.values[:10]

In [ ]:
insincere_df = raw_df[raw_df.target == 1]

In [ ]:
insincere_df.question_text.values[:10]

In [ ]:
raw_df.target.value_counts(normalize=True)

In [ ]:
raw_df.target.value_counts(normalize=True).plot(kind='bar')

In [ ]:
test_df=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
test_df

In [ ]:
sub_df=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')
sub_df

In [ ]:
sub_df.prediction.value_counts()

## Text Preprocessing Techniques
Outline:
1. Understand the bag of words model
2. Tokenization
3. Stop word removal
4. Stemming

## Bag of Words Intuition
Create a list of all the words across all the text documents
You convert each document into vector counts of each word

In [ ]:
q0 = sincere_df.question_text.values[1]

In [ ]:
q0

In [ ]:
q1 = raw_df[raw_df.target == 1].question_text.values[0]

In [ ]:
q1

## Tokenization
splitting a document into words and separators

In [ ]:
import nltk

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')

In [ ]:
q0

In [ ]:
word_tokenize(q0)

In [ ]:
word_tokenize(' this is (something) with, a lot of, punctuation;')

In [ ]:
q1

In [ ]:
word_tokenize(q1)

In [ ]:
q0_tok = word_tokenize(q0)
q1_tok = word_tokenize(q1)

## Stop Word Removal
Removing commonly occuring words

In [ ]:
q1_tok

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
english_stopwords = stopwords.words('english')

In [ ]:
", ".join(english_stopwords)

In [ ]:
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in english_stopwords]

In [ ]:
q0_tok

In [ ]:
q0_stp = remove_stopwords(q0_tok)

In [ ]:
q0_stp

In [ ]:
q1_stp = remove_stopwords(q1_tok)

In [ ]:
q1_tok

In [ ]:
q1_stp

## Stemming
"go", "gone", "going" -> "go"
"birds", "bird" -> "bird"

In [ ]:
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer(language='english')

In [ ]:
stemmer.stem('going')

In [ ]:
stemmer.stem('supposedly')


In [ ]:
q0_stm = [stemmer.stem(word) for word in q0_stp]

In [ ]:
q0_stm 

In [ ]:
q1_stm = [stemmer.stem(word) for word in q1_stp]

In [ ]:
q1_stp

In [ ]:
q1_stm

## Lemmatization
"love" -> "love"
"loving" -> "love"
"lovable" -> "love"

Implement Bag of Words

Outline:

1. Create a vocabulary using Count Vectorizer
2. Transform text to vectors using Count Vectorizer
3. Configure text preprocessing in Count Vectorizer

## Create a Vocabulary

Create a Working Sample


In [ ]:
import os

In [ ]:
IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

In [ ]:
if IS_KAGGLE:
    SAMPLE_SIZE = len(raw_df)
else:
    SAMPLE_SIZE = 100_000

In [ ]:
sample_df = raw_df.sample(SAMPLE_SIZE, random_state=42)

In [ ]:
sample_df

In [ ]:
small_df = sample_df[:5]

In [ ]:
small_df

In [ ]:
small_df.question_text.values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
small_vect = CountVectorizer()

In [ ]:
small_vect.fit(small_df.question_text)

In [ ]:
small_vect.get_feature_names_out()

## Transform documents into Vectors

In [ ]:
vectors = small_vect.transform(small_df.question_text)

In [ ]:
vectors

In [ ]:
vectors.shape

In [ ]:
small_df.question_text.values[0]

In [ ]:
vectors[0].toarray()

In [ ]:
vectors.toarray()

## Configure Count Vectorizer Parameters

In [ ]:
stemmer = SnowballStemmer(language='english')

In [ ]:
def tokenize(text):
    return [stemmer.stem(word) for word in word_tokenize(text)]

In [ ]:
tokenize('What is the really (dealing) here?')


In [ ]:
vectorizer = CountVectorizer(lowercase=True, 
                             tokenizer=None,
                             stop_words=english_stopwords,
                             max_features=1000)

In [ ]:
vectorizer.fit(sample_df.question_text)

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
vectorizer.get_feature_names_out()[:100]

In [ ]:
inputs = vectorizer.transform(sample_df.question_text)

In [ ]:
inputs.shape


In [ ]:
inputs

In [ ]:
sample_df.question_text.values[0]

In [ ]:
test_df

In [ ]:
test_inputs = vectorizer.transform(test_df.question_text)

## ML Models for Text Classification
 Outline:
1. Create a training & validation set
2. Train a logistic regression model
3. Make predictions on training, validation & test data

### Split into Training and Validation Set

In [ ]:
sample_df

In [ ]:
inputs.shape


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
train_inputs, val_inputs, train_targets, val_targets = train_test_split(inputs, sample_df.target, 
                                                                        test_size=0.3, random_state=42)

In [ ]:
train_inputs.shape

In [ ]:
train_targets.shape

In [ ]:
val_inputs.shape

In [ ]:
val_targets.shape

#### Train Logistic Regression model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
MAX_ITER = 1000

In [ ]:
model = LogisticRegression(max_iter=MAX_ITER, solver='sag')

In [ ]:
model.fit(train_inputs, train_targets)


## Make predictions using the model

In [ ]:
train_preds = model.predict(train_inputs)

In [ ]:
train_targets


In [ ]:
train_preds

In [ ]:
pd.Series(train_preds).value_counts()

In [ ]:
pd.Series(train_targets).value_counts()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(train_targets, train_preds)

In [ ]:
accuracy_score(train_targets, np.zeros(len(train_targets)))

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(train_targets, train_preds)

In [ ]:
f1_score(train_targets, np.zeros(len(train_targets)))

In [ ]:
random_preds = np.random.choice((0, 1), len(train_targets))
f1_score(train_targets, random_preds)

In [ ]:
val_preds = model.predict(val_inputs)

In [ ]:
accuracy_score(val_targets, val_preds)

In [ ]:
f1_score(val_targets, val_preds)

In [ ]:
sincere_df.question_text.values[:10]

In [ ]:
sincere_df.target.values[:10]

In [ ]:
model.predict(vectorizer.transform(sincere_df.question_text.values[:10]))

In [ ]:
insincere_df.question_text.values[:10]

In [ ]:
insincere_df.target.values[:10]

In [ ]:
model.predict(vectorizer.transform(insincere_df.question_text.values[:10]))

##  Make Predictions and Submit to Kaggle

In [ ]:
test_df

In [ ]:
test_inputs.shape

In [ ]:
test_preds = model.predict(test_inputs)

In [ ]:
test_preds = model.predict(test_inputs)

In [ ]:
sub_df

In [ ]:
sub_df.prediction = test_preds

In [ ]:
sub_df.prediction.value_counts()

In [ ]:
sub_df

In [ ]:
sub_df

In [ ]:
sub_df.to_csv('submission.csv', index=None)

In [ ]:
!head submission.csv